## Initializing & Training with SpanMarker
[SpanMarker](https://github.com/tomaarsen/SpanMarkerNER) is an accessible yet powerful Python module for training Named Entity Recognition models.

In this short notebook, we'll have a look at how to initialize and train an NER model using SpanMarker. For a larger and more general tutorial on how to use SpanMarker, please have a look at the [Getting Started](getting_started.ipynb) notebook.

### Setup
First of all, the `span_marker` Python module needs to be installed. If we want to use [Weights and Biases](https://wandb.ai/) for logging, we can install `span_marker` using the `[wandb]` extra.

In [ ]:
%pip install span_marker
# %pip install span_marker[wandb]

### Loading the dataset
For this example, we'll load the commonly used [CoNLL2003 dataset](https://huggingface.co/datasets/conll2003) from the Hugging Face hub using 🤗 Datasets.

In [2]:
from datasets import load_dataset

dataset = load_dataset("conll2003")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
labels = dataset["train"].features["ner_tags"].feature.names
labels

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

SpanMarker accepts any dataset as long as it has `tokens` and `ner_tags` columns. The `ner_tags` can be annotated using the IOB, IOB2, BIOES or BILOU labeling scheme, but also regular unschemed labels. This CoNLL dataset uses the common IOB or IOB2 labeling scheme, with PER, ORG, LOC and MISC labels.

### Initializing a `SpanMarkerModel`
A SpanMarker model is initialized via [SpanMarkerModel.from_pretrained](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.modeling.html#span_marker.modeling.SpanMarkerModel.from_pretrained). This method will be familiar to those who know 🤗 Transformers. It accepts either a path to a local model or the name of a model on the [Hugging Face Hub](https://huggingface.co/models).

Importantly, the model can *either* be an encoder or an already trained and saved SpanMarker model. As we haven't trained anything yet, we will use an encoder. To learn how to load and use a saved SpanMarker model, please have a look at the [Loading & Inferencing](model_loading.ipynb) notebook.

Reasonable options for encoders include BERT and RoBERTa, which means that the following are all good options:

* [prajjwal1/bert-tiny](https://huggingface.co/prajjwal1/bert-tiny)
* [prajjwal1/bert-mini](https://huggingface.co/prajjwal1/bert-mini)
* [prajjwal1/bert-small](https://huggingface.co/prajjwal1/bert-small)
* [prajjwal1/bert-medium](https://huggingface.co/prajjwal1/bert-medium)
* [bert-base-cased](https://huggingface.co/bert-base-cased)
* [bert-large-cased](https://huggingface.co/bert-large-cased)
* [roberta-base](https://huggingface.co/roberta-base)
* [roberta-large](https://huggingface.co/roberta-large)

Not all encoders work though, they **must** allow for `position_ids` as an input argument, which disqualifies DistilBERT, T5, DistilRoBERTa, ALBERT & BART. Furthermore, using uncased models is generally not recommended, as the capitalisation can be very useful to find named entities.

We'll use `"roberta-base"` for this notebook. If you're running this on Google Colab, be sure to set hardware accelerator to "GPU" in `Runtime` > `Change runtime type`.

In [4]:
from span_marker import SpanMarkerModel

model_name = "roberta-base"
model = SpanMarkerModel.from_pretrained(
    model_name,
    labels=labels,
    model_max_length=256,
    entity_max_length=6,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


For us, these warnings are expected, as we are initializing `BertModel` for a new task.

Note that we provided [SpanMarkerModel.from_pretrained](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.modeling.html#span_marker.modeling.SpanMarkerModel.from_pretrained) with a list of our labels. This is required when training a new model. See [Configuring](model_configuration.ipynb) for more details and recommendations on configuration options.

### Training
At this point, our model is already ready for training! We can import [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) directly from 🤗 Transformers as well as the SpanMarker `Trainer`. The `Trainer` is a subclass of the 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) that simplifies some tasks for you, but otherwise it works just like the regular `Trainer`.

This next snippet shows some reasonable defaults. Feel free to adjust the batch size to a lower value if you experience out of memory exceptions.

In [5]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="models/span-marker-roberta-base-conll03",
    learning_rate=1e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    push_to_hub=False,
    logging_steps=50,
    fp16=True,
    warmup_ratio=0.1,
)

Now we can create a SpanMarker `Trainer` in the same way that you would initialize a 🤗 Transformers `Trainer`. We'll train on a subsection of the data to save us some time. Amazingly, this `Trainer` will automatically create logs using exactly the logging tools that you have installed. With other words, if you prefer logging with [Tensorboard](https://www.tensorflow.org/tensorboard), all that you have to do is install it.

In [6]:
from span_marker import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"].select(range(2000)),
)
trainer.train()

This SpanMarker model will ignore 0.097877% of all annotated entities in the train dataset. This is caused by the SpanMarkerModel maximum entity length of 6 words.
These are the frequencies of the missed entities due to maximum entity length out of 23499 total entities:
- 18 missed entities with 7 words (0.076599%)
- 2 missed entities with 8 words (0.008511%)
- 3 missed entities with 10 words (0.012767%)


  0%|          | 0/1802 [00:00<?, ?it/s]

{'loss': 0.9477, 'learning_rate': 2.6519337016574586e-06, 'epoch': 0.03}
{'loss': 0.2025, 'learning_rate': 5.414364640883978e-06, 'epoch': 0.06}
{'loss': 0.1407, 'learning_rate': 8.176795580110498e-06, 'epoch': 0.08}
{'loss': 0.1291, 'learning_rate': 9.895126465144972e-06, 'epoch': 0.11}
{'loss': 0.0973, 'learning_rate': 9.58667489204195e-06, 'epoch': 0.14}
{'loss': 0.0737, 'learning_rate': 9.278223318938926e-06, 'epoch': 0.17}
{'loss': 0.0639, 'learning_rate': 8.969771745835904e-06, 'epoch': 0.19}
{'loss': 0.0539, 'learning_rate': 8.661320172732882e-06, 'epoch': 0.22}
{'loss': 0.0481, 'learning_rate': 8.352868599629858e-06, 'epoch': 0.25}
{'loss': 0.0489, 'learning_rate': 8.044417026526836e-06, 'epoch': 0.28}


This SpanMarker model won't be able to predict 0.172563% of all annotated entities in the evaluation dataset. This is caused by the SpanMarkerModel maximum entity length of 6 words.
These are the frequencies of the missed entities due to maximum entity length out of 3477 total entities:
- 5 missed entities with 7 words (0.143802%)
- 1 missed entities with 10 words (0.028760%)


{'eval_loss': 0.03809420391917229, 'eval_overall_precision': 0.8559068219633943, 'eval_overall_recall': 0.7527070529704419, 'eval_overall_f1': 0.8009965742759265, 'eval_overall_accuracy': 0.9548683524504692, 'eval_runtime': 13.4517, 'eval_samples_per_second': 153.661, 'eval_steps_per_second': 38.434, 'epoch': 0.28}
{'loss': 0.0379, 'learning_rate': 7.735965453423812e-06, 'epoch': 0.31}
{'loss': 0.039, 'learning_rate': 7.42751388032079e-06, 'epoch': 0.33}
{'loss': 0.0373, 'learning_rate': 7.119062307217768e-06, 'epoch': 0.36}
{'loss': 0.0362, 'learning_rate': 6.810610734114744e-06, 'epoch': 0.39}
{'loss': 0.0287, 'learning_rate': 6.502159161011722e-06, 'epoch': 0.42}
{'loss': 0.0283, 'learning_rate': 6.193707587908698e-06, 'epoch': 0.44}
{'loss': 0.0308, 'learning_rate': 5.885256014805676e-06, 'epoch': 0.47}
{'loss': 0.0266, 'learning_rate': 5.576804441702654e-06, 'epoch': 0.5}
{'loss': 0.0193, 'learning_rate': 5.26835286859963e-06, 'epoch': 0.53}
{'loss': 0.0163, 'learning_rate': 4.959

Loading cached processed dataset at ...
Loading cached processed dataset at ...


{'loss': 0.0178, 'learning_rate': 1.8753855644663791e-06, 'epoch': 0.83}


{'eval_loss': 0.013812492601573467, 'eval_overall_precision': 0.9375370041444642, 'eval_overall_recall': 0.9268364062042728, 'eval_overall_f1': 0.9321559970566594, 'eval_overall_accuracy': 0.9858902502606882, 'eval_runtime': 13.6173, 'eval_samples_per_second': 151.793, 'eval_steps_per_second': 37.967, 'epoch': 0.83}
{'loss': 0.017, 'learning_rate': 1.566933991363356e-06, 'epoch': 0.86}
{'loss': 0.0164, 'learning_rate': 1.2584824182603333e-06, 'epoch': 0.89}
{'loss': 0.0202, 'learning_rate': 9.500308451573104e-07, 'epoch': 0.92}
{'loss': 0.0203, 'learning_rate': 6.415792720542875e-07, 'epoch': 0.94}
{'loss': 0.0188, 'learning_rate': 3.3312769895126467e-07, 'epoch': 0.97}
{'loss': 0.0175, 'learning_rate': 2.4676125848241828e-08, 'epoch': 1.0}
{'train_runtime': 326.1576, 'train_samples_per_second': 44.193, 'train_steps_per_second': 5.525, 'train_loss': 0.06725485075976323, 'epoch': 1.0}


And now the final step is to compute the model's performance.

In [7]:
metrics = trainer.evaluate()
metrics

Loading cached processed dataset at ...
Loading cached processed dataset at ...


{'eval_loss': 0.01344103179872036,
 'eval_overall_precision': 0.9364892678623934,
 'eval_overall_recall': 0.9321041849575651,
 'eval_overall_f1': 0.9342915811088296,
 'eval_overall_accuracy': 0.9861183524504692,
 'eval_runtime': 13.1532,
 'eval_samples_per_second': 157.148,
 'eval_steps_per_second': 39.306,
 'epoch': 1.0}

Additionally, we should evaluate using the test set.

In [8]:
trainer.evaluate(dataset["test"], metric_key_prefix="test")

{'test_loss': 0.027700792998075485,
 'test_overall_precision': 0.9039692701664532,
 'test_overall_recall': 0.9067889908256881,
 'test_overall_f1': 0.9053769350554182,
 'test_overall_accuracy': 0.9796867082332724,
 'test_runtime': 22.7367,
 'test_samples_per_second': 155.915,
 'test_steps_per_second': 39.012,
 'epoch': 1.0}

Once trained, we can save our new model locally.

In [9]:
trainer.save_model("models/span-marker-roberta-base-conll03/checkpoint-final")

Or we can push it to the 🤗 Hub like so. I've commented it away for now to prevent people from accidentally pushing models.

In [10]:
# trainer.push_to_hub()

If we want to use it again, we can just load it using the checkpoint or using the model name on the Hub. This is how it would be done using a local checkpoint. See the [Loading & Inferencing](model_loading.ipynb) notebook for more details.

In [11]:
# model = SpanMarkerModel.from_pretrained("models/span-marker-roberta-base-conll03/checkpoint-final")

That was all! As simple as that. If we put it all together into a single script, it looks something like this:
```python
from datasets import load_dataset
from span_marker import SpanMarkerModel, Trainer
from transformers import TrainingArguments

dataset = load_dataset("conll2003")
labels = dataset["train"].features["ner_tags"].feature.names

model_name = "roberta-base"
model = SpanMarkerModel.from_pretrained(model_name, labels=labels, model_max_length=256)

args = TrainingArguments(
    output_dir="models/span-marker-roberta-base-conll03",
    learning_rate=1e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    push_to_hub=False,
    logging_steps=50,
    warmup_ratio=0.1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"].select(range(8000)),
    eval_dataset=dataset["validation"].select(range(2000)),
)

trainer.train()
trainer.save_model("models/span-marker-roberta-base-conll03/checkpoint-final")
trainer.push_to_hub()

metrics = trainer.evaluate()
print(metrics)
```

With `wandb` initialized, you can enjoy their very useful training graphs straight in your browser. It ends up looking something like this.
![image](https://user-images.githubusercontent.com/37621491/235172501-a3cdae91-faf0-42b7-ac60-e6738b78e67e.png)
![image](https://user-images.githubusercontent.com/37621491/235172726-795ded55-4b1c-40fa-ab91-476762f7dd57.png)

Furthermore, you can use the `wandb` hyperparameter search functionality using the tutorial from the Hugging Face documentation [here](https://huggingface.co/docs/transformers/hpo_train). This transfers very well to the SpanMarker `Trainer`.